### How to use

Example
```
get_period("01.01.2019", "02.01.2019", "EURUSD")
```
* start - string, format "dd.mm.yyyy"
* end - string, format "dd.mm.yyyy"
* instrument - string, Forex currency pair

In [23]:
import os
import pandas
from io import StringIO
from urllib.request import urlopen
from datetime import datetime, timedelta

def download_day(date, instrument):
    url = (
        "http://export.finam.ru/{0}-{1}-{2}-{3}.txt?"
        "market=5&"
        "em=83&"
        "code={0}&"
        "apply=0&"
        "df={1}&"
        "mf={5}&"
        "yf={3}&"
        "from={4}&"
        "dt={1}&"
        "mt={5}&"
        "yt={3}&"
        "to={4}&"
        "p=2&"
        "f={0}-{1}-{2}-{3}&"
        "e=.txt&"
        "cn={0}&"
        "dtf=1&"
        "tmf=1&"
        "MSOR=1&"
        "mstime=on&"
        "mstimever=1&"
        "sep=1&"
        "sep2=1&"
        "datf=1&"
        "at=1"
    ).format(instrument, date.day, date.month, date.year, date.strftime('%d.%m.%Y'), date.month - 1)
    print(url)
    response = urlopen(url)
    csv = response.read()

    file_name = "data/{}-{}.csv".format(instrument, date.strftime('%d-%m-%Y'))
    with open(file_name, "wb") as file:
        file.write(csv)
    
    return csv_to_dataframe(csv)


def load_day_from_file(date, instrument):
    file_name = "data/{}-{}.csv".format(instrument, date.strftime('%d-%m-%Y'))
    if (os.path.isfile(file_name)):
        with open(file_name, "r") as file:
            data = file.read()
            return csv_to_dataframe(data)
    return None


def csv_to_dataframe(csv):
    return pandas.read_csv(StringIO(csv), sep=",")


def get_period(start, end, instrument):
    start_date = datetime.strptime(start, "%d.%m.%Y")
    end_date = datetime.strptime(end, "%d.%m.%Y")
    days_count = int((end_date - start_date).days) + 1
    data_list = []

    for day in range(days_count):
        date = start_date + timedelta(day)
        file_contents = load_day_from_file(date, instrument)
        if (file_contents is None):
            data_list.append(download_day(date, instrument))
        else:
            data_list.append(file_contents)

    return data_list